In [ ]:
API_URL = "localhost:8099"

In [38]:
MCP_URL = f"{API_URL}/google-maps/mcp"

In [ ]:
!pip install google-genai

In [40]:
import requests
import json
from google import genai
import os
from IPython.display import display, Markdown

In [41]:
# Set up Gemini API
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")
client = genai.Client(api_key=GEMINI_API_KEY)

In [42]:
def call_maps_tool(query):
    """Call the Google Maps MCP tool with a specific query."""
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "tools/call",
        "params": {
            "name": "maps_search_places",
            "arguments": {
                "query": query
            }
        }
    }

    try:
        response = requests.post(MCP_URL, json=payload)
        response.raise_for_status()

        parsed_response = json.loads(response.text)
        places_data = None

        # Extract the places data from the nested response
        for item in parsed_response["result"]["content"]:
            if item["type"] == "text":
                places_data = json.loads(item["text"])
                break

        return places_data
    except Exception as e:
        print(f"Error calling Maps tool: {e}")
        return None

In [43]:
def should_use_maps_tool(query):
    """Use Gemini to decide if the query requires the Maps tool."""
    prompt = f"""
    Determine if the following query requires information about places, locations, or maps:

    Query: "{query}"

    Respond with just "YES" if the query needs geographical information like:
    - Finding places (restaurants, hotels, attractions, etc.)
    - Information about locations
    - Distances between places
    - Addresses or directions

    Otherwise respond with just "NO".
    """

    response = client.models.generate_content(
        model="gemini-2.0-flash",  # or "gemini-1.0-pro" depending on what's available to you
        contents=[prompt]
    )
    decision = response.text.strip().upper()
    return decision == "YES"

In [44]:
def answer_with_gemini(query, places_data=None):
    """Generate a response using Gemini with optional places data."""

    if places_data:
        # Format the first 5 places to avoid overwhelming the model
        places_info = []
        for i, place in enumerate(places_data.get("places", [])[:5]):
            places_info.append(f"""
            Place {i+1}: {place.get('name')}
            Address: {place.get('formatted_address')}
            Rating: {place.get('rating', 'N/A')}
            Types: {', '.join(place.get('types', []))}
            """)

        places_text = "\n".join(places_info)

        prompt = f"""
        The user asked: "{query}"

        I searched for places and found the following information:

        {places_text}

        Please provide a helpful response to the user's query using this information. Include specific details about the places when relevant.
        """
    else:
        prompt = f"""
        The user asked: "{query}"

        Please provide a helpful response to this query. If you don't have enough information, you can suggest the user to be more specific.
        """

    response = client.models.generate_content(
        model="gemini-2.0-flash",  # or "gemini-1.0-pro"
        contents=[prompt]
    )
    return response.text

In [45]:
def process_user_query(query):
    """Process a user query using the appropriate tools."""
    # Decide if we should use the Maps tool
    use_maps = should_use_maps_tool(query)

    if use_maps:
        print("Using Google Maps tool to find information...")
        places_data = call_maps_tool(query)
        if places_data:
            response = answer_with_gemini(query, places_data)
        else:
            response = "I couldn't retrieve location information. Please try again."
    else:
        print("Answering without using Maps tool...")
        response = answer_with_gemini(query)

    return response

In [46]:
def run_assistant():
    user_query = input("What would you like to know? ")
    response = process_user_query(user_query)

    try:
        # Try using IPython display (works in notebooks)
        display(Markdown(response))
    except (NameError, TypeError):
        # Fallback for terminal environments
        print("\n--- Response ---")
        print(response)
        print("--------------\n")

In [47]:
# Run the assistant
run_assistant()

What would you like to know? coffee shops in france
Using Google Maps tool to find information...


Okay, I found some highly-rated coffee shops in Paris, France:

*   **Clove:** Located at 14 Rue Chappe in the 18th arrondissement, it boasts a rating of 4.9.
*   **The Caféothèque of Paris:** This cafe, with a rating of 4.3, is located at 52 Rue de l'Hôtel de ville in the 4th arrondissement.
*   **Crible - coffee shop:** Find this cafe at 75 Rue Buffon in the 5th arrondissement. It also has a rating of 4.9.
*   **BlackBird Coffee:** Located at 77 Rue du Temple in the 3rd arrondissement, it has a rating of 4.7. It's categorized as both a cafe and a restaurant.
*   **Coffee Specialty Coffee:** This cafe, with a rating of 4.9, is located at 40 Rue Chapon in the 3rd arrondissement.

All of these options are categorized as cafes and have food available, so you should be able to grab a coffee and a bite to eat. Enjoy!
